## Packages

In [1]:
import pandas as pd
from glob import glob
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

## Import DF

In [2]:
gastos = ['%deEjecución',
 '(USDpercapitanominal)',
 'Clasificaciónfuncional(%delPIB',
 'Clasificaciónfuncional(%delTot']



for gasto in gastos:
    #### Interpolation and Moving Average Imputation

    writer = pd.ExcelWriter(f'0_INPUT//{gasto}.xlsx', engine='xlsxwriter')

    dfg = pd.read_excel("0_INPUT//INPUT.xlsx", sheet_name=f"{gasto}")
    dfg = dfg.replace(0, np.nan)


    def impute_interpolate(series):
        return series.interpolate(limit_direction='both')
    dfg_imputed = dfg.groupby('Country Code').apply(impute_interpolate)


    def fillna_with_moving_average(group, window):
        return group.fillna(group.rolling(window, min_periods=1).mean())
    grouped = dfg_imputed.groupby('Country Code')
    dfg_imputed = grouped.apply(lambda group: fillna_with_moving_average(group, window=3))

    dfg_imputed.to_excel(writer, sheet_name="INTERPOLATION", index = False)


    #### Multivariate chains
    # Create the IterativeImputer object
    imputer = IterativeImputer(max_iter=10, random_state=0)

    # Define the columns to exclude during imputation
    exclude_columns = ['Time', 'Country Code']

    # Select only the columns with data that you want to impute
    df_to_impute_ce = dfg_imputed.drop(columns=exclude_columns)

    # Identify and store the columns with all values as NaN
    nan_cols = df_to_impute_ce.columns[df_to_impute_ce.isna().all()].tolist()
    nan_cols_positions = [df_to_impute_ce.columns.get_loc(col) for col in nan_cols]

    # Remove the columns with all values as NaN
    df_to_impute_ce = df_to_impute_ce.dropna(axis=1, how='all')

    # Fit and transform the data for imputation
    df_imputed_ce = imputer.fit_transform(df_to_impute_ce)

    # Convert the resulting array back to a DataFrame, using only the imputed columns
    df_imputed_ce = pd.DataFrame(df_imputed_ce, columns=df_to_impute_ce.columns)

    # Insert back the columns with all values as NaN in their original positions
    for col, pos in zip(nan_cols, nan_cols_positions):
        df_imputed_ce.insert(pos, col, dfg_imputed[col])

    # Reinsert the panel identification columns
    df_imputed_ce[exclude_columns] = dfg_imputed[exclude_columns]
    df_imputed_ce[nan_cols] = dfg_imputed[nan_cols]
    df_imputed_ce = df_imputed_ce[list(dfg_imputed.columns)]
    

    # Seleccionar las celdas con valores entre 200 y 1000 y dividirlos por 10

    if gasto == '%deEjecución':
    
        # Obtener las columnas que deben ser transformadas (excluyendo 'Time' y 'Country Code')
        columns_to_transform = [col for col in df_imputed_ce.columns if col not in ['Time', 'Country Code']]

        # Seleccionar las celdas con valores entre 200 y 1000 y dividirlos por 10
        mask1 = (df_imputed_ce[columns_to_transform] > 200) & (df_imputed_ce[columns_to_transform] <= 1000)
        for column in columns_to_transform:
            df_imputed_ce.loc[mask1[column], column] /= 10

        # Seleccionar las celdas con valores mayores a 1000 y dividirlos por 100
        mask2 = df_imputed_ce[columns_to_transform] > 1000
        for column in columns_to_transform:
            df_imputed_ce.loc[mask2[column], column] /= 100



    df_imputed_ce.to_excel(writer, sheet_name="CHAINS", index = False)

    #### KNN
    # Create the KNNImputer object with desired settings
    imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean', copy=True, add_indicator=False)

    # Define the columns to exclude during imputation
    exclude_columns = ['Time', 'Country Code']

    # Identify and store the columns with all values as NaN
    nan_cols = dfg_imputed.columns[dfg_imputed.isna().all()].tolist()

    # Select only the columns with data to impute
    df_to_impute = dfg_imputed.drop(columns=exclude_columns + nan_cols)

    # Remove the columns with all values as NaN
    df_to_impute = df_to_impute.dropna(axis=1, how='all')

    # Perform imputation by fitting and transforming the data
    df_imputed = pd.DataFrame(imputer.fit_transform(df_to_impute), columns=df_to_impute.columns)

    # Reinsert the excluded columns and rearrange the columns to match the original DataFrame
    df_imputed[exclude_columns] = dfg_imputed[exclude_columns]
    df_imputed[nan_cols] = dfg_imputed[nan_cols]
    df_imputed = df_imputed[list(dfg_imputed.columns)]


    # Obtener las columnas que deben ser transformadas (excluyendo 'Time' y 'Country Code')
    columns_to_transform = [col for col in df_imputed.columns if col not in ['Time', 'Country Code']]


    if gasto == '%deEjecución':

        # Seleccionar las celdas con valores entre 200 y 1000 y dividirlos por 10
        mask1 = (df_imputed[columns_to_transform] > 200) & (df_imputed[columns_to_transform] <= 1000)
        for column in columns_to_transform:
            df_imputed.loc[mask1[column], column] /= 10

        # Seleccionar las celdas con valores mayores a 1000 y dividirlos por 100
        mask2 = df_imputed[columns_to_transform] > 1000
        for column in columns_to_transform:
            df_imputed.loc[mask2[column], column] /= 100

    
    df_imputed.to_excel(writer, sheet_name="KNN", index = False)


    writer.close()


<ipython-input-2-1a9e6611e4c9>:19: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dfg_imputed = dfg.groupby('Country Code').apply(impute_interpolate)
<ipython-input-2-1a9e6611e4c9>:23: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Country Code'], dtype='object')
  return group.fillna(group.rolling(window, min_periods=1).mean())
<ipython-input-2-1a9e6611e4c9>:25: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys w

# Summary


# Interpolation and Moving Average Imputation

This procedure involves two main steps: interpolation and filling with a moving average. 

## Interpolation

The first step is interpolation, which is used to estimate unknown values that fall between two known values. It's performed on each country group separately. 

Mathematically, linear interpolation between two points $(x_1, y_1)$ and $(x_2, y_2)$ for a given $x$ is given by the formula:

$$y = y_1 + \frac{(y_2 - y_1)}{(x_2 - x_1)}(x - x_1)$$

## Moving Average Imputation

The second step involves replacing remaining missing values with a moving average. A simple moving average of a time series $x_1, x_2, ..., x_n$ over a fixed window size $w$ is a series of unweighted means of the previous $w$ data. 

The formula for the moving average $MA$ over window size $w$ is:

$$MA = \frac{1}{w}(x_1 + x_2 + ... + x_w)$$

This means we calculate the mean of the preceding $w$ data points. If a data point is missing in this window, it will be filled with the moving average calculated over this window. This process is repeated for all data points in the time series, and for each country separately, taking into account the panel data structure.
"""

# Multivariate Imputation by Chained Equations (MICE)

### 1. Initialization

In the initialization step, we fill missing values with a simple statistic such as the mean. If we represent our data as a matrix $X$ of size $n$ x $m$ (where $n$ is the number of samples and $m$ is the number of features), and we denote by $X_{missing}$ the subset of $X$ that contains missing values, we could write:

$$
X_{missing}^{(0)} = mean(X_{observed})
$$

Where $X_{missing}^{(0)}$ represents the initial imputed values for the missing data, and $mean(X_{observed})$ is the mean of the observed (non-missing) values in $X$.

### 2. Imputation

For each feature $X_i$ with missing data, we create a regression model using the observed part of $X_i$ as the target variable and all the other features (with the imputed values for the current iteration for the missing parts) as the input variables. Let's denote by $X_{-i}$ the matrix obtained from $X$ by removing the column $i$. The prediction step could be written as follows:

$$
X_{i,missing}^{(k)} = model(X_{-i,observed}, X_{i,observed}).predict(X_{-i,missing})
$$

Where:
- $X_{i,missing}^{(k)}$ is the imputed values for the missing part of feature $X_i$ at iteration $k$.
- $model(X_{-i,observed}, X_{i,observed})$ is a model (such as a linear regression or a decision tree) trained on the observed part of the data.
- $X_{-i,missing}$ is the part of the data that is used to make predictions for the missing values in $X_i$.

### 3. Iteration

We repeat the imputation step a fixed number of times $K$, to build variability in the imputations.

### 4. Analysis

After the final round of imputation, we analyze each "completed" dataset separately. This could be any type of analysis, from simple descriptive statistics to complex machine learning models.

### 5. Pooling

The results of the multiple analyses are combined into one final result. This is typically done using Rubin's rules, which compute the final estimate as the mean of the estimates from each imputed dataset, and the variance as a combination of the within-imputation variance (the average of the variances from each imputed dataset) and the between-imputation variance (the variance of the estimates from each imputed dataset).

For an estimate $\theta$ computed on each imputed dataset, we have:

$$
\bar{\theta} = \frac{1}{K}\sum_{k=1}^{K}\theta^{(k)}
$$

$$
Var(\bar{\theta}) = \frac{1}{K}\sum_{k=1}^{K}Var(\theta^{(k)}) + \frac{1 + 1/K}{K} \sum_{k=1}^{K}(\theta^{(k)} - \bar{\theta})^2
$$


## K-Nearest Neighbors (KNN) Imputation

K-Nearest Neighbors (KNN) imputation is a method used to fill missing values in a dataset by estimating them based on the values of their neighboring data points. It is a non-parametric imputation technique that relies on similarity between samples to impute missing values. Here's a step-by-step explanation of the KNN imputation process:

1. Determine the Number of Neighbors (K)

   - The first step in KNN imputation is to determine the number of neighbors (K) to consider when imputing a missing value. The choice of K is crucial and depends on the dataset and problem at hand. It is typically selected based on empirical evaluation or domain knowledge.

2. Find the K Nearest Neighbors

   - For each sample with missing values, the algorithm identifies the K nearest neighbors based on some similarity metric (e.g., Euclidean distance, Manhattan distance). The neighbors are selected from the samples with complete information for the feature being imputed.

   The KNN similarity equation can be written as:

   $$
   \text{Similarity} = \frac{1}{1 + d(X_i, X_j)}
   $$

   Where:
   - $X_i$ represents the sample with missing values.
   - $X_j$ represents the samples with complete information.
   - $d$ is the similarity metric (e.g., Euclidean distance, Manhattan distance).

3. Impute the Missing Value

   - Once the K nearest neighbors are identified, the missing value is imputed by aggregating the values of the corresponding feature from the neighbors. The imputation can be done by taking the mean, median, or some other measure of central tendency of the neighbor values.

   The imputation equation can be written as:

   $$
   X_{\text{missing}}^{(0)} = \text{mean}(X_{\text{observed}})
   $$

   Where:
   - $X_{\text{missing}}^{(0)}$ represents the initial imputed values for the missing data.
   - $\text{mean}(X_{\text{observed}})$ is the mean of the observed (non-missing) values in $X$.



| Imputation Method                                 | Advantages                                                                                                                                     | Disadvantages                                                                                                              |
| ------------------------------------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------- |
| Multivariate Imputation by Chained Equations (MICE) | - Handles missing data in multiple variables simultaneously                                                                                    | - Requires specifying the imputation model for each variable                                                              |
|                                                  | - Preserves the correlations between variables during imputation                                                                                | - Relatively computationally expensive                                                                                      |
|                                                  | - Can handle different types of variables (e.g., continuous, categorical)                                                                      | - May introduce additional variability due to the random imputation process                                                  |
|                                                  | - Provides estimates of uncertainty through multiple imputations and pooling techniques (e.g., Rubin's rules)                                    | - Assumes Missing at Random (MAR) mechanism                                                                                  |
|                                                  | - Can be used with various imputation models (e.g., linear regression, decision trees)                                                         |                                                                                                                            |
|                                                  | - Allows flexibility in handling non-linear relationships and interactions between variables                                                  |                                                                                                                            |
| K-Nearest Neighbors (KNN) Imputation                 | - Simple and intuitive approach                                                                                                                | - Requires determining the number of neighbors (K)                                                                           |
|                                                  | - Preserves local data patterns and relationships during imputation                                                                             | - Sensitive to the choice of similarity metric and distance measure                                                         |
|                                                  | - Can handle different types of variables (e.g., continuous, categorical)                                                                      | - Computationally expensive for large datasets or high-dimensional data                                                     |
|                                                  | - Does not assume any specific distribution or model for the data                                                                                | - May produce biased imputations if the nearest neighbors are not truly similar to the missing sample                        |
|                                                  | - Can be combined with other imputation methods or used as a preprocessing step                                                                 | - Missingness in one variable may affect the imputations of other variables due to their interdependencies                  |
|                                                  | - Allows flexibility in choosing the imputation strategy (e.g., mean, median, mode, etc.)                                                      |                                                                                                                            |
